In [3]:
"""
Author: Daniel Wu
Purpose: PS5 - Train a HMM model
         on tagged italian and japanese text         
output: hmmmodel.txt        
"""

import os
import sys
import math
import re
import string
import glob
import numpy as np
import random
import json

def pause():
    programPause = input("Press the <ENTER> key to continue...")
    print("Paused Program")
    
# root_dir = sys.argv[1]
root_dir = "/Users/user/Desktop/Fall_2020/CSCI_544/Coding_Assignments/PA5/hmm-training-data/it_isdt_train_tagged.txt"

with open(root_dir) as fp:
    data = fp.readlines()    

# Build transition (A) and emission (B) "matrices" 
A_dic = {}
B_dic = {}

# Counts 
for line in data:
    words = line.split(" ")
    words[-1] = words[-1].rstrip("\n")    
    
    #q_0 start
    tag_0 = "q_0"
    
    for word in words:
        token, tag_1 = word.rsplit('/',1)
        
        if tag_0 not in A_dic:
            A_dic[tag_0] = {}
        if tag_1 not in B_dic:
            B_dic[tag_1] = {}
        
        if tag_1 not in A_dic[tag_0]:            
            A_dic[tag_0][tag_1] = 1
        elif tag_1 in A_dic[tag_0]:
            A_dic[tag_0][tag_1] += 1
        
        if token not in B_dic[tag_1]:
            B_dic[tag_1][token] = 1
        elif token in B_dic[tag_1]:
            B_dic[tag_1][token] += 1
        
        tag_0 = tag_1
        
# create and smooth prior probs
for p in A_dic:
    if p == 'q_0': continue
    if p not in A_dic['q_0']:
        A_dic['q_0'][p] = 0.000001
    elif p in A_dic['q_0']:
        A_dic['q_0'][p] += 0.000001  

priors = {k:math.log(v/sum(A_dic['q_0'].values())) for (k,v) in A_dic['q_0'].items()}
        
# create and smooth transition probs
A_Mtx = {}

for tag_0 in A_dic: 
    if tag_0 == 'q_0': continue
        
    for tag_1 in A_dic:        
        if tag_0 == 'q_0': continue
            
        if tag_1 not in A_dic[tag_0]:
            A_dic[tag_0][tag_1] = 0.000001
        elif tag_1 in A_dic[tag_0]:
            A_dic[tag_0][tag_1] += 0.000001
        
for tag in A_dic:
    if tag == 'q_0': continue
    A_Mtx[tag] = {k:math.log(v/sum(A_dic[tag].values())) for (k,v) in A_dic[tag].items()}

# create emission probs (no smooth)
B_Mtx = {}
for tag in B_dic:
    B_Mtx[tag] = {k:math.log(v/sum(B_dic[tag].values())) for (k,v) in B_dic[tag].items()}


master_dic = {}
master_dic['priors'] = priors
master_dic['A_Mtx'] = A_Mtx
master_dic['B_Mtx'] = B_Mtx
    
#JSON outfile
file = open("hmmmodel.txt", 'w', encoding = 'utf-8')
json.dump(master_dic, file)
file.close()